In [1]:
import pyterrier as pt
import json
from NIRfunction import *
import pyterrier as pt
import os

pt.init(
        mem=8000,
        version="snapshot",
        tqdm="notebook",
        boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"],
    )

Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by jitpack on 2022-04-12 09:28)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# iter_indexer = pt.IterDictIndexer(
#     '/content/drive/MyDrive/NIR/NIR2022 dataset/indexes/iterindex',
#     overwrite=True,
#     meta=["docno", "title", "text"],
#     meta_lengths=[20, 100, 4096],
# )

# indexref = iter_indexer.index(generate_dataset(), fields=["title", "text"])
# index = pt.IndexFactory.of(indexref)

index = pt.IndexFactory.of("/Users/zhouyuqin/github/NIR/Project/indexes/stage2/iterindex")

print(index.getCollectionStatistics())

Number of documents: 528155
Number of terms: 520520
Number of postings: 83716130
Number of fields: 2
Number of tokens: 145322007
Field names: [title, text]
Positions:   false



In [3]:
train_query = get_query('../Project/NIR2022 dataset/data/train_query.csv')
train_qrel = get_qrel('../Project/NIR2022 dataset/data/train_qrel.csv')

In [4]:
BM25_qe = pt.BatchRetrieve(index, wmodel="BM25", controls={"qemodel" : "Bo2", "qe" : "on", "c": 0.5, "bm25.k_1": 0.9})

#### Query expansion using Glove pre-trained embedding

In [5]:
!pip install --upgrade gensim

In [6]:
import gensim
import gensim.downloader as api

In [7]:
# Download and load Glove vectors
model = api.load('glove-wiki-gigaword-100')
print(model.most_similar('tree'))

[('trees', 0.8510393500328064), ('grass', 0.6922112703323364), ('pine', 0.6856961846351624), ('oak', 0.6387961506843567), ('leaf', 0.6364573836326599), ('bushes', 0.6336526870727539), ('flower', 0.6273626089096069), ('green', 0.6112821698188782), ('garden', 0.6033942103385925), ('bark', 0.6031672358512878)]


In [8]:
# Expand each query by adding the top-2 similar words for each word in the query
k = 2

topics_qe_df = train_query.copy()
for i in range(len(topics_qe_df)):
    q = topics_qe_df.iloc[i]['query']
    qe = []
    for word in q.split(' '):
        expanded_words = [pair[0] for pair in model.most_similar(word, topn=k)]
        expanded_words.append(word)
        qe.append(expanded_words)
    topics_qe_df.iloc[i]['query'] = gensim.parsing.preprocessing.remove_stopwords(" ".join([e for l in qe for e in l]))

topics_qe_df

,qid,query
0,301,european global international organised organi...
1,302,"polio measles poliomyelitis , office times pos..."
2,303,telescope nasa hubble telescopes hubble telesc...
3,306,africa africans african military civilians civ...
4,307,york new hydropower hydroelectricity hydroelec...
...,...,...
195,693,newspaper dailies newspapers digital electroni...
196,695,black brown white black shirt collar criminal ...
197,697,force aircraft air congestion rail traffic con...
198,698,numeracy education literacy rate inflation rat...


In [9]:
# Evaluate previous systems on the expanded topics
pt.Experiment(
    retr_systems=[BM25_qe],
    topics=topics_qe_df,
    qrels=train_qrel,
    eval_metrics=["map", "ndcg_cut_5", "ndcg_cut_10", "ndcg_cut_20"])

,name,map,ndcg_cut_5,ndcg_cut_10,ndcg_cut_20
0,BR(BM25),0.190875,0.349375,0.333815,0.317989


In [10]:
# Expand each query by adding the top-5 similar words for each word in the query
k = 5

topics_qe_df = train_query.copy()
for i in range(len(topics_qe_df)):
    q = topics_qe_df.iloc[i]['query']
    qe = []
    for word in q.split(' '):
        expanded_words = [pair[0] for pair in model.most_similar(word, topn=k)]
        expanded_words.append(word)
        qe.append(expanded_words)
    topics_qe_df.iloc[i]['query'] = gensim.parsing.preprocessing.remove_stopwords(" ".join([e for l in qe for e in l]))

topics_qe_df

,qid,query
0,301,european global world nations regional interna...
1,302,polio measles diphtheria chickenpox rabies pol...
2,303,telescope nasa orbiting shuttle spacecraft hub...
3,306,africa africans asian south nations african mi...
4,307,york new hydropower hydroelectricity hydro dam...
...,...,...
195,693,newspaper dailies magazines news publications ...
196,695,black brown gray red green white black shirt b...
197,697,force aircraft planes flying jet air congestio...
198,698,numeracy education educational illiteracy math...


In [11]:
### to replace "/"
topics_qe_df.loc[134, "query"] = 'killings arrest detentions raids suspects arrests bombings attacks attack suicide raid bombing fod 4c khobar skyscraper wtc'
topics_qe_df.loc[197, "query"] = "force aircraft planes flying jet air congestion rail train roads passenger traffic controllers interface device sensor controller"

In [13]:
# Evaluate previous systems on the expanded topics
pt.Experiment(
    retr_systems=[BM25_qe],
    topics=topics_qe_df,
    qrels=train_qrel,
    eval_metrics=["map", "ndcg_cut_5", "ndcg_cut_10", "ndcg_cut_20"])

,name,map,ndcg_cut_5,ndcg_cut_10,ndcg_cut_20
0,BR(BM25),0.143248,0.253845,0.250397,0.241389


#### Cross-encoder (with or without segments)
The implementation code of this part is completed by colab  which can be found in this link: [colab](https://colab.research.google.com/drive/1Q1P1Q4YYn9_Po0kh2vVyJD_g9K3kOeoH?usp=sharing).   
Here I just load their train outcome, or output test files


Models name: re-rank 1000

"app526app000"  = 'dph (kl)'  
"app526app001"  = 'BM25 (Bo2)'  
"app526app002"  = 'BM25 >> Cross-Encoder (without segments)' 

(max)  
"app526app003" = 'BM25 >> Cross-Encoder (with segments)' (strid = 3, length = 15)  
"app526app004" = 'BM25 >> Cross-Encoder (with segments)' (strid = 3, length = 10)  
"app526app005" = 'BM25 >> Cross-Encoder (with segments)' (strid = 3, length = 6)  

(mean)
"app526app006" = 'BM25 >> Cross-Encoder (with segments)' (strid = 3, length = 6)

##### Results of trainning set

In [135]:
DPH_qe = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "KL", "qe" : "on"})
BM25_qe = pt.BatchRetrieve(index, wmodel="BM25", controls={"qemodel" : "Bo2", "qe" : "on", "c": 0.5, "bm25.k_1": 0.9})   # tuned parameters 
metrics = {"map", "ndcg_cut_5", "ndcg_cut_10", "ndcg_cut_20"}

In [137]:
experiment = pt.Experiment(
    retr_systems=[DPH_qe, BM25_qe],
    names=["DPH_qe", 'BM25_qe'],
    topics= train_query,
    qrels= train_qrel,
    eval_metrics=metrics)

In [161]:
import pytrec_eval

# Load qrels in a dictionary
qrels_dict = dict()
for _, r in train_qrel.iterrows():
    qid, docno, label = r
    if qid not in qrels_dict:
        qrels_dict[qid] = dict()
    qrels_dict[qid][docno] = int(label)

# Build evaluator based on the qrels and metrics
my_qrel = {q: d for q, d in qrels_dict.items()}
evaluator = pytrec_eval.RelevanceEvaluator(my_qrel, metrics)

In [162]:
# Load run
with open("/Users/zhouyuqin/github/NIR/Project/NIR2022 dataset/outputs/train0 copy.run", 'r') as f_run:
    cross_run0 = pytrec_eval.parse_run(f_run)

# Evaluate Cross-Encoder model
cross_evals0 = evaluator.evaluate(cross_run0)

# Compute performance in different metrics for each query
cross_metric2vals0 = {m: [] for m in metrics}
for q, d in cross_evals0.items():
    for m, val in d.items():
        cross_metric2vals0[m].append(val)

# Average results by query
cross_metric2avg0 = dict()
for m in metrics:
    val = pytrec_eval.compute_aggregated_measure(m, cross_metric2vals0[m])
    cross_metric2avg0[m] = val
    print(m, '\t', val)

ndcg_cut_5 	 0.45784561272991975
map 	 0.2375310958891805
ndcg_cut_10 	 0.43650327260040184
ndcg_cut_20 	 0.41345836644852124


In [142]:
cross_metric2avg0['name'] = 'BM25 >> Cross-Encoder0 (wthout segments)'
experiment = experiment.append(cross_metric2avg0, ignore_index=True)
experiment

/var/folders/1c/95b9dlpd2xv9yjh6v0xw08700000gn/T/ipykernel_19518/1485129171.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  experiment = experiment.append(cross_metric2avg0, ignore_index=True)


,name,ndcg_cut_5,map,ndcg_cut_10,ndcg_cut_20
0,DPH_qe,0.485266,0.291812,0.470110,0.453887
1,BM25_qe,0.477150,0.293812,0.463044,0.442785
2,BM25 >> Cross-Encoder0 (wthout segments,0.457846,0.237531,0.436503,0.413458


In [276]:
import pytrec_eval

# Load qrels in a dictionary
qrels_dict = dict()
for _, r in train_qrel.iterrows():
    qid, docno, label = r
    if qid not in qrels_dict:
        qrels_dict[qid] = dict()
    qrels_dict[qid][docno] = int(label)

# Build evaluator based on the qrels and metrics
my_qrel = {q: d for q, d in qrels_dict.items()}
evaluator = pytrec_eval.RelevanceEvaluator(my_qrel, metrics)

In [146]:
# Load run
with open("/Users/zhouyuqin/github/NIR/Project/NIR2022 dataset/outputs/train1.run", 'r') as f_run:
    cross_run1 = pytrec_eval.parse_run(f_run)

# Evaluate Cross-Encoder model
cross_evals1 = evaluator.evaluate(cross_run1)

# Compute performance in different metrics for each query
cross_metric2vals1 = {m: [] for m in metrics}
for q, d in cross_evals1.items():
    for m, val in d.items():
        cross_metric2vals1[m].append(val)

# Average results by query
cross_metric2avg1 = dict()
for m in metrics:
    val = pytrec_eval.compute_aggregated_measure(m, cross_metric2vals1[m])
    cross_metric2avg1[m] = val
    print(m, '\t', val)

ndcg_cut_5 	 0.5022852672374377
map 	 0.3041166948593583
ndcg_cut_10 	 0.5004037043939596
ndcg_cut_20 	 0.47926621440747263


In [147]:
cross_metric2avg1['name'] = 'BM25 >> Cross-Encoder (with segments)'
experiment = experiment.append(cross_metric2avg1, ignore_index=True)
experiment

/var/folders/1c/95b9dlpd2xv9yjh6v0xw08700000gn/T/ipykernel_19518/1401061753.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  experiment = experiment.append(cross_metric2avg1, ignore_index=True)


,name,ndcg_cut_5,map,ndcg_cut_10,ndcg_cut_20
0,DPH_qe,0.485266,0.291812,0.470110,0.453887
1,BM25_qe,0.477150,0.293812,0.463044,0.442785
2,BM25 >> Cross-Encoder0 (wthout segments,0.457846,0.237531,0.436503,0.413458
3,BM25 >> Cross-Encoder (with segments),0.502285,0.304117,0.500404,0.479266


##### Test Output


In [270]:
with open("../Project/NIR2022 dataset/outputs/testsum.run") as f:
    file = f.readlines()

In [271]:
with open("../Project/NIR2022 dataset/outputs/testsum.run", "w") as f:
    for i in file:
        a = f'{i.split(" ")[0]} Q0 {i.split(" ")[2]} {int(i.split(" ")[3])+1 } {i.split(" ")[4]} app526APP006'
        f.write(a +"\n")